In [159]:
import geocoder
import requests
import pandas as pd
import numpy as np
import time
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

pd.set_option('max_colwidth', 200)

In [38]:
"""
ЗАВАНТАЖЕННЯ ПІДГОТОЛВЕНИХ ДАНИХ ЗІ ШКОЛАМИ ТА АДРЕСАМИ ДЛЯ ОПРАЦЮВАННЯ
"""

df_map_pivot = pd.read_csv('24_regions.csv', index_col = 0, encoding='UTF-8')
df_reborn = pd.read_csv('minosvity_fixed.csv', index_col = 0) #"Довідник шкіл з адресами"


"""
МЕРДЖЕННЯ
"""


df_map_pivot = df_map_pivot.merge(df_reborn, left_on = 'Заклад', right_on = 'city_fixed')
df_map_pivot = df_map_pivot[['Заклад', 'Витрачена сума', 'address_fixed', 'full_3']]

In [89]:
"""
ФУНКЦІЯ ДЛЯ ВИЗНАЧЕННЯ КООРДИНАТ ЗАКЛАДІВ
"""

def get_coordinates(df):
    
    geolocator = Nominatim(user_agent = "specify_your_app_name_here")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 1)

#     df["Coordinates"] = df["Заклад"].apply(geocode) #визначило 75% закладів
#     df["Coordinates"] = df["full_3"].apply(geocode) 
    df["Coordinates"] = df["address_temp"].apply(geocode)
    df["Latitude"] = df["Coordinates"].apply(lambda x: x.latitude if x != None else None)
    df["Longitude"] = df["Coordinates"].apply(lambda x: x.longitude if x != None else None)
    
    return df

In [55]:
# df_map_pivot = get_coordinates(df_map_pivot)
df_map_pivot.to_csv('school_coordinates.csv')
print("Не вдалось визначти:", df_map_pivot.Latitude.isnull().sum())
df_map_no_error = df_map_pivot[df_map_pivot['Latitude'].notnull()]
print(len(df_map_no_error))
df_map_no_error.to_csv('school_coordinates_success.csv')

1436

## Problem Coordinates

In [56]:
df_map_error = df_map_pivot[df_map_pivot['Latitude'].isnull()]

543

In [73]:
df_map_error_try1 = get_coordinates(df_map_error) #спроби визначити координати за іншими колонками
df_map_error_try2 = df_map_error_try1[df_map_error_try1['Latitude'].isnull()]
df_map_error_try1 = df_map_error_try1[df_map_error_try1['Latitude'].notnull()]
df_map_error_try1.to_csv('school_coordinates_try1.csv')

In [75]:
df_map_error_try3 = get_coordinates(df_map_error_try2)
print(len(df_map_error_try3))
print("Не вдалось визначти:", df_map_error_try3.Latitude.isnull().sum())

522
Не вдалось визначти: 486


C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\user1\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [76]:
df_map_error_try4 = df_map_error_try3[df_map_error_try3['Latitude'].isnull()]
df_map_error_try3 = df_map_error_try3[df_map_error_try3['Latitude'].notnull()]
df_map_error_try3.to_csv('school_coordinates_try2.csv')

In [94]:
df_with_coordinates = pd.concat([df_map_error_try3, df_map_error_try1, df_map_no_error])

In [97]:
df_without_coordinates = df_map_error_try4.copy()

In [99]:
df_with_coordinates_temp = df_with_coordinates[['Заклад', 'Latitude', 'Longitude', 'full_3']]

In [100]:
df_with_coordinates_temp.columns

Index(['Заклад', 'Latitude', 'Longitude', 'full_3'], dtype='object')

## Merge with procurement database

In [102]:
df_procurements = pd.read_csv('24_regions.csv', index_col = 0)
df_procurements = df_procurements[['Заклад', 'Витрачена сума', 'Посилання']]
print(len(df_procurements))

3954


In [103]:
df_ready = pd.merge(df_procurements, df_with_coordinates_temp, how='inner', on="Заклад",
         left_index=False, right_index= False, sort = True)

In [106]:
df_ready = df_ready.rename(columns={"full_3": "Повна адреса"})

,Заклад,Витрачена сума,Посилання,Latitude,Longitude,Повна адреса
0,Івано-Франківськ ДНЗ №3,"1,757,670.030000",https://dozorro.org/tender/UA-2017-07-13-00057...,48.922522,24.710319,м. Івано-Франківськ вул. Набережна імені Вас...
1,Івано-Франківськ Школа №1,"7,834,080.920000",https://dozorro.org/tender/UA-2017-05-22-00128...,48.927835,24.703678,м. Івано-Франківськ вул. Калуське шосе 1
2,Івано-Франківськ Школа №1,"7,834,080.920000",https://dozorro.org/tender/UA-2017-05-22-00128...,48.927835,24.703678,м. Івано-Франківськ вул. Довга 37
3,Івано-Франківськ Школа №10,"715,729.420000",https://dozorro.org/tender/UA-2018-06-11-00249...,48.938842,24.744832,м. Івано-Франківськ вул. Вовчинецька 196 б
4,Івано-Франківськ Школа №11,"1,281,345.940000",https://dozorro.org/tender/UA-2017-07-17-00155...,48.920018,24.718487,м. Івано-Франківськ вул. Богдана Лепкого 9
5,Івано-Франківськ Школа №12,"731,301.000000",https://dozorro.org/tender/UA-2018-11-09-00029...,48.910691,24.706599,м. Івано-Франківськ вул. Національної Гварді...
6,Івано-Франківськ Школа №15,"307,550.580000",https://dozorro.org/tender/UA-2018-07-16-00122...,48.916740,24.735820,м. Івано-Франківськ вул. Незалежності 207
7,Івано-Франківськ Школа №16,"4,193,498.400000",https://dozorro.org/tender/UA-2017-06-26-00186...,48.930727,24.724468,м. Івано-Франківськ вул. Вовчинецька 103
8,Івано-Франківськ Школа №17,"288,800.000000",https://dozorro.org/tender/UA-2017-10-03-00105...,48.933470,24.699833,м. Івано-Франківськ вул. Набережна імені В.С...
9,Івано-Франківськ Школа №18,"645,305.980000",https://dozorro.org/tender/UA-2018-08-20-00116...,48.945866,24.699223,м. Івано-Франківськ вул. Тролейбусна 7


In [155]:
df_ready['Витрачена сума'] = df_ready['Витрачена сума'].astype('int64', copy=False)

In [157]:
df_ready.to_excel('for_map.xls')

In [161]:
df_ready.sample(10)

,Заклад,Витрачена сума,Посилання,Latitude,Longitude,Повна адреса
733,Львів Школа №27,7308443,"https://dozorro.org/tender/UA-2017-05-25-000260-c, https://dozorro.org/tender/UA-2017-08-09-001939-b, https://dozorro.org/tender/UA-2018-12-11-001143-b",49.825457,24.036066,м. Львів вул. Іл. Свєнціцького 15
46,Вінниця Школа №30,1086600,"https://dozorro.org/tender/UA-2017-08-11-001518-b, https://dozorro.org/tender/UA-2018-07-26-001428-b, https://dozorro.org/tender/UA-2018-07-20-000813-b, https://dozorro.org/tender/UA-2018-09-17-00...",49.248520,28.501260,м. Вінниця вул. Стрілецька 62
39,Вінниця Школа №22,3443839,"https://dozorro.org/tender/UA-2017-07-24-001017-b, https://dozorro.org/tender/UA-2017-06-13-001143-b, https://dozorro.org/tender/UA-2017-08-11-000263-b, https://dozorro.org/tender/UA-2018-07-23-00...",49.229881,28.465419,м. Вінниця Староміський район міста вул. Данила Нечая 21
1051,Суми Школа №15,2647590,"https://dozorro.org/tender/UA-2017-05-16-001926-b, https://dozorro.org/tender/UA-2017-06-09-000430-b, https://dozorro.org/tender/UA-2017-09-04-000637-a, https://dozorro.org/tender/UA-2017-11-09-00...",50.912020,34.778555,м. Суми вул. Пушкіна 56
1264,Харків Школа №66,1083920,"https://dozorro.org/tender/UA-2018-06-22-000968-a, https://dozorro.org/tender/UA-2019-09-04-001226-b, https://dozorro.org/tender/UA-2019-08-29-000153-c, https://dozorro.org/tender/UA-2019-07-09-00...",50.029852,36.228061,м. Харків Основ'янський район міста просп. Гагаріна 260
262,Запоріжжя Школа №45,210396,"https://dozorro.org/tender/UA-2018-07-10-001069-c, https://dozorro.org/tender/UA-2019-08-15-000955-a, https://dozorro.org/tender/UA-2019-04-18-001565-b",47.817411,35.033328,м. Запоріжжя Хортицький район міста просп. Ювілейний 30 - А
1341,Херсон Школа №50,1867084,"https://dozorro.org/tender/UA-2017-11-10-001849-b, https://dozorro.org/tender/UA-2017-05-13-000543-b, https://dozorro.org/tender/UA-2018-03-22-000164-b, https://dozorro.org/tender/UA-2018-03-20-00...",46.659170,32.642265,м. Херсон Дніпровський район міста вул. Кримська 135
839,Миколаїв Школа №45,20015410,"https://dozorro.org/tender/UA-2017-11-24-000310-b, https://dozorro.org/tender/UA-2017-04-27-001018-b, https://dozorro.org/tender/UA-2017-05-25-001101-b, https://dozorro.org/tender/UA-2017-12-12-00...",46.956650,32.062768,м. Миколаїв Інгульський район міста вул. 4-а Поздовжня 58
1321,Херсон Школа №28,1066813,"https://dozorro.org/tender/UA-2017-11-10-000120-b, https://dozorro.org/tender/UA-2017-04-20-000232-b, https://dozorro.org/tender/UA-2017-11-23-002534-a, https://dozorro.org/tender/UA-2018-02-09-00...",46.673915,32.605981,м. Херсон пров. Маяковського 1
281,Запоріжжя Школа №67,853705,"https://dozorro.org/tender/UA-2017-08-03-000609-a, https://dozorro.org/tender/UA-2017-09-06-001839-c, https://dozorro.org/tender/UA-2018-07-30-000832-c, https://dozorro.org/tender/UA-2018-07-19-00...",47.889215,35.147412,м. Запоріжжя Заводський район міста вул. Радіаторна 26
